In [208]:
from dotenv import load_dotenv
load_dotenv()

import os
import json
import re
from typing import TypedDict, List, Dict, Optional
from langgraph.graph import StateGraph, END
import google.generativeai as genai
from tavily import TavilyClient


In [209]:
gemini_api_key = os.getenv("GEMINI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")


In [210]:

if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY not found in environment variables!")
genai.configure(api_key=gemini_api_key)

if not tavily_api_key:
    raise ValueError("TAVILY_API_KEY not found in environment variables!")
tavily = TavilyClient(api_key=tavily_api_key)


In [211]:

# ---- Helper Functions ----
def call_gemini(prompt: str, retries=3, delay=5) -> str:
    """Call Gemini API with retry logic."""
    for attempt in range(retries):
        try:
            model = genai.GenerativeModel("gemini-2.5-flash")  # Stable model
            response = model.generate_content(prompt)
            return response.text.strip() if response.text else ""
        except Exception as e:
            print(f"Gemini API error (attempt {attempt+1}/{retries}): {e}")
            if attempt < retries - 1:
                import time
                time.sleep(delay)
            else:
                return ""


In [212]:

def search_tavily(query: str) -> list[str]:
    """Return up to 5 Tavily search result snippets as a list of strings."""
    try:
        response = tavily.search(query=query[:370], top_k=5)
        results = response.get("results", [])
        return [r.get("snippet") or r.get("content") or r.get("title", "") for r in results][:5]
    except Exception as e:
        print(f"Tavily search failed: {e}")
        return []

In [213]:


def safe_parse_json(response_text: str, fallback: Dict = None) -> Dict:
    """Safely parse JSON - optimized version."""
    if not response_text or not response_text.strip():
        return fallback or {"rating": 0, "feedback": "Empty response from API"}
    
    # Try to extract JSON using regex (this is what's actually working)
    match = re.search(r'\{.*\}', response_text, re.DOTALL)
    if match:
        try:
            return json.loads(match.group(0))
        except json.JSONDecodeError:
            pass
    
    return fallback or {"rating": 0, "feedback": "Failed to parse JSON response"}

In [214]:
# Run this cell separately first to upload CV
def upload_cv_interactively():
    """Upload CV separately before starting the interview graph"""
    print(" Welcome to the AI Interviewer (Question Evaluation Mode)!")
    print("Please upload your CV/Resume")
    
    # Create file upload widget for CV
    file_upload = widgets.FileUpload(
        accept='.pdf,.txt,.doc,.docx',
        multiple=False,
        description='Upload CV/Resume'
    )
    
    upload_button = widgets.Button(description="Process CV")
    output = widgets.Output()
    
    cv_data = {}
    
    def on_upload_button_clicked(b):
        with output:
            output.clear_output()
            if file_upload.value:
                # Handle both tuple format (newer ipywidgets) and dict format (older)
                if isinstance(file_upload.value, tuple):
                    uploaded_file = file_upload.value[0]
                else:
                    uploaded_file = list(file_upload.value.values())[0]
                
                filename = uploaded_file['name']
                content = uploaded_file['content']
                
                print(f"📄 Processing CV: {filename}")
                
                # Extract text from CV
                cv_text = extract_text_from_cv(filename, content)
                
                if not cv_text.strip():
                    print("❌ Could not extract text from the CV file")
                    return
                
                # Analyze CV to determine interview topic
                topic = analyze_cv_for_topic(cv_text)
                
                print(f"🎯 Detected interview focus: {topic}")
                
                cv_data.update({
                    "cv_content": cv_text,
                    "topic": topic,
                    "cv_filename": filename
                })
                
                print("✅ CV processed successfully!")
                print("📝 You can now run the interview with the processed CV data.")
                
            else:
                print("❌ Please upload a CV file first!")
    
    # Display upload interface
    upload_widget = widgets.VBox([
        widgets.HBox([file_upload, upload_button]),
        output
    ])
    display(upload_widget)
    upload_button.on_click(on_upload_button_clicked)
    
    return cv_data



In [215]:

class InterviewState(TypedDict):
    topic: str
    content: List[str]
    questions: List[str]
    answers: List[str]
    feedback: List[Dict]
    current_question: Optional[str]
    current_answer: Optional[str]
    step: int
    max_questions: int
    final_evaluation: Optional[Dict]
    messages: List[Dict]
    question_type: str
    cv_content: str  # Add this
    cv_filename: str  # Add this

In [216]:
def upload_and_process_cv():
    """Handle CV upload using file upload widget."""
    print(" Welcome to the AI Interviewer (Question Evaluation Mode)!")
    print("Please upload your CV/Resume")
    
    # Create file upload widget for CV
    file_upload = widgets.FileUpload(
        accept='.pdf,.txt,.doc,.docx',
        multiple=False,
        description='Upload CV/Resume'
    )
    
    upload_button = widgets.Button(description="Process CV")
    output = widgets.Output()
    
    cv_data = {}
    processing_complete = False
    
    def on_upload_button_clicked(b):
        nonlocal processing_complete
        with output:
            output.clear_output()
            if file_upload.value:
                # Handle both tuple format (newer ipywidgets) and dict format (older)
                if isinstance(file_upload.value, tuple):
                    # Newer ipywidgets - value is a tuple of file info
                    uploaded_file = file_upload.value[0]
                    filename = uploaded_file['name']
                    content = uploaded_file['content']
                else:
                    # Older ipywidgets - value is a dict
                    uploaded_file = list(file_upload.value.values())[0]
                    filename = uploaded_file['name']
                    content = uploaded_file['content']
                
                print(f"📄 Processing CV: {filename}")
                
                # Extract text from CV
                cv_text = extract_text_from_cv(filename, content)
                
                if not cv_text.strip():
                    print("❌ Could not extract text from the CV file")
                    return
                
                # Analyze CV to determine interview topic
                topic = analyze_cv_for_topic(cv_text)
                
                print(f"🎯 Detected interview focus: {topic}")
                
                cv_data.update({
                    "cv_content": cv_text,
                    "topic": topic,
                    "cv_filename": filename
                })
                
                print("✅ CV processed successfully! Ready to start interview.")
                processing_complete = True
            else:
                print("❌ Please upload a CV file first!")
    
    # Display upload interface
    upload_widget = widgets.VBox([
        widgets.HBox([file_upload, upload_button]),
        output
    ])
    display(upload_widget)
    upload_button.on_click(on_upload_button_clicked)
    
    # Wait for processing to complete
    print("⏳ Waiting for CV upload...")
    while not processing_complete:
        import time
        time.sleep(1)
    
    return cv_data



def extract_text_from_cv(filename: str, content: bytes) -> str:
    """Extract text from CV based on file type."""
    try:
        if filename.endswith('.pdf'):
            return extract_text_from_pdf(content)
        elif filename.endswith('.txt'):
            return content.decode('utf-8')
        elif filename.endswith(('.doc', '.docx')):
            return extract_text_from_doc(content, filename)
        else:
            # Fallback: try to decode as text
            return content.decode('utf-8', errors='ignore')
    except Exception as e:
        print(f"Error extracting text from CV: {e}")
        return ""

def extract_text_from_pdf(content: bytes) -> str:
    """Extract text from PDF content."""
    try:
        import PyPDF2
        pdf_file = io.BytesIO(content)
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        return text
    except ImportError:
        print("PyPDF2 not available. Install with: pip install PyPDF2")
        return "PDF text extraction requires PyPDF2"
    except Exception as e:
        print(f"PDF extraction error: {e}")
        return ""

def extract_text_from_doc(content: bytes, filename: str) -> str:
    """Extract text from Word documents."""
    try:
        if filename.endswith('.docx'):
            from docx import Document
            doc_file = io.BytesIO(content)
            doc = Document(doc_file)
            return "\n".join([paragraph.text for paragraph in doc.paragraphs])
        else:
            # For .doc files, you might need antiword or other tools
            return "Word document extraction limited for .docx files"
    except ImportError:
        print("python-docx not available. Install with: pip install python-docx")
        return "DOCX extraction requires python-docx"

def analyze_cv_for_topic(cv_text: str) -> str:
    """Analyze CV content to determine interview focus."""
    prompt = f"""
    Analyze this CV/resume and determine the most appropriate technical interview topic focus.
    Consider skills, experience, technologies mentioned, and role preferences.
    
    CV Content:
    {cv_text[:3000]}  # Limit length for API
    
    Return ONLY the main interview topic as a short phrase (e.g., "Python backend development", "Data science with machine learning", "Frontend React development", "C++ systems programming").
    """
    
    topic = call_gemini(prompt) or "Software Development"
    return topic.strip()

def generate_first_question_from_cv(cv_text: str, topic: str, question_type: str) -> str:
    """Generate first question based on CV content."""
    prompt = f"""
    Based on the following CV and the interview topic '{topic}', generate an appropriate first interview question.
    
    CV Excerpt:
    {cv_text[:2000]}
    
    Question Style: {'Broad, general question' if question_type.startswith('broad') else 'Specific, detailed question'}
    
    The question should be relevant to the candidate's background and the topic '{topic}'.
    Return ONLY the question text.
    """
    
    question = call_gemini(prompt) or f"Tell me about your experience with {topic}."
    return question

In [217]:

def setup_node(state: InterviewState) -> InterviewState:
    print(" Welcome to the AI Interviewer (Question Evaluation Mode)!")
    
    # CV content should already be in state
    cv_content = state.get("cv_content", "")
    topic = state.get("topic", "")
    
    if not cv_content:
        raise ValueError("No CV content found. Please upload a CV first.")
    
    print(f"🎯 Interview focus: {topic}")
    print("Choose question style:")
    print("1. Broad, follow-up questions (general, builds on previous answers)")
    print("2. Narrow, follow-up questions (specific, probes details from previous answers)")
    print("3. Broad, non-follow-up questions (general, new topic aspects)")
    print("4. Narrow, non-follow-up questions (specific, new topic aspects)")
    choice = input("Enter choice (1-4): ").strip()
    question_type_map = {
        "1": "broad_followup",
        "2": "narrow_followup",
        "3": "broad_nonfollowup",
        "4": "narrow_nonfollowup"
    }
    question_type = question_type_map.get(choice, "broad_followup")

    content_list = search_tavily(f"technical interview questions for {topic} role")
    initial_messages = [{"role": "user", "content": f"CV-based interview for: {topic}"}]

    # Generate the first question based on CV
    first_question = generate_first_question_from_cv(cv_content, topic, question_type)

    return {
        **state,
        "content": content_list,
        "messages": initial_messages,
        "step": 0,
        "questions": [],
        "answers": [],
        "feedback": [],
        "current_question": first_question,
        "question_type": question_type
    }

In [218]:

def get_answer_node(state: InterviewState) -> InterviewState:
    """Get candidate answer"""
    current_q = state.get("current_question")
    if not current_q:
        raise ValueError("No current_question found in state.")

    print(f"\n❓ Generated Question {state['step'] + 1}: {current_q}\n")
    answer = input("💭 Your answer: ").strip()

    new_messages = state['messages'] + [
        {"role": "interviewer", "content": current_q},
        {"role": "candidate", "content": answer}
    ]

    return {
        **state,
        "current_answer": answer,
        "messages": new_messages,
        "questions": state['questions'] + [current_q],
        "answers": state['answers'] + [answer]
    }

def evaluate_question_node(state: InterviewState) -> InterviewState:
    """Rate the quality of the last question and the candidate's answer."""
    # Compile transcript of previous Q&A and feedback
    transcript = ""
    for i in range(len(state['questions']) - 1):
        q = state['questions'][i]
        a = state['answers'][i]
        f = state['feedback'][i] if i < len(state['feedback']) else {}
        transcript += f"Previous Q{i+1}: {q}\nPrevious A{i+1}: {a}\nPrevious Feedback: {f.get('question_feedback', {}).get('feedback', '')}\n\n"

    last_q = state['questions'][-1]
    last_a = state['answers'][-1]
    full_messages = json.dumps(state['messages'])
    full_content = "\n".join(state['content'])

    # Question feedback prompt
    question_prompt = f"""
You are an expert interviewer. Evaluate the following question for its clarity, relevance, and ability to probe understanding, considering the ENTIRE interview history, accumulated context, all previous messages, questions, answers, and feedback.

Full Interview History (Messages): {full_messages}
Accumulated Context (Search Snippets): {full_content}
Previous Q&A Transcript: {transcript}
Current Question: {last_q}
Current Candidate Answer: {last_a}

Provide a rating (1-10) for question quality and 2-3 sentence feedback. Consider how well this question builds on prior answers, avoids repetition, incorporates context, and advances the topic.
Return in JSON format:
{{
    "rating": 0,
    "feedback": "..."
}}
"""
    question_feedback_text = call_gemini(question_prompt)
    question_feedback = safe_parse_json(question_feedback_text, {"rating": 0, "feedback": "Failed to generate question feedback."})

    # Answer feedback prompt
    answer_prompt = f"""
You are an expert interviewer. Evaluate the following candidate answer for its clarity, relevance, depth, and alignment with the question, considering the ENTIRE interview history and context.

Full Interview History (Messages): {full_messages}
Accumulated Context (Search Snippets): {full_content}
Previous Q&A Transcript: {transcript}
Current Question: {last_q}
Current Candidate Answer: {last_a}

Provide a rating (1-10) for answer quality and 2-3 sentence feedback. Highlight strengths and areas for improvement.
Return in JSON format:
{{
    "rating": 0,
    "feedback": "..."
}}
"""
    answer_feedback_text = call_gemini(answer_prompt)
    answer_feedback = safe_parse_json(answer_feedback_text, {"rating": 0, "feedback": "Failed to generate answer feedback."})

    feedback = {
        "question_feedback": question_feedback,
        "answer_feedback": answer_feedback
    }
    # print(f"💡 Question Feedback: {question_feedback['feedback']} (Rating: {question_feedback['rating']})")
    # print(f"💡 Answer Feedback: {answer_feedback['feedback']} (Rating: {answer_feedback['rating']})")

    return {
        **state,
        "feedback": state['feedback'] + [feedback],
        "step": state['step'] + 1
    }

In [219]:

def generate_question_node(state: InterviewState) -> InterviewState:
    """Generate next question based on type (broad/narrow, follow-up/non-follow-up)."""
    updated_content = state['content']
    question_type = state['question_type']
    is_followup = "followup" in question_type
    is_broad = question_type.startswith("broad")

    if state['step'] > 0:
        last_q = state['questions'][-1]
        last_a = state['answers'][-1]
        tavily_results = search_tavily(f"{state['topic']} interview context: Q: {last_q} A: {last_a}")
        updated_content += tavily_results

    prompt_instruction = ""
    if is_followup:
        prompt_instruction = f"Generate a {'broad, general' if is_broad else 'specific, detailed'} follow-up question that directly probes details from the previous answer: {state['answers'][-1] if state['answers'] else ''}."
    else:
        prompt_instruction = f"Generate a {'broad, general' if is_broad else 'specific, detailed'} question that explores a new aspect of the topic, independent of the previous answer."

    prompt_question = f"""
You are an expert interviewer. Using the following reference content:
{updated_content}

{prompt_instruction}
Topic: {state['topic']}
Question number: {state['step'] + 1}
Return ONLY the question text.
"""
    question = call_gemini(prompt_question) or f"Tell me more about {state['topic']}."

    return {
        **state,
        "current_question": question,
        "content": updated_content
    }


In [220]:

def final_evaluation_node(state: InterviewState) -> InterviewState:
    """Generate final evaluation of all questions."""
    print("\n📊 Generating final evaluation of all questions...")
    transcript = ""
    for i, (q, a, f) in enumerate(zip(state['questions'], state['answers'], state['feedback']), 1):
        transcript += f"Q{i}: {q}\nA{i}: {a}\nQuestion Feedback: {f['question_feedback']['feedback']} (Rating: {f['question_feedback']['rating']})\nAnswer Feedback: {f['answer_feedback']['feedback']} (Rating: {f['answer_feedback']['rating']})\n\n"

    prompt = f"""
Based on this transcript, produce a JSON summary evaluation of the questions:
{transcript}

JSON format ONLY:
{{
    "overall_quality": 0-10,
    "strengths": ["..."],
    "areas_for_improvement": ["..."],
    "recommendation": "keep/revise/remove",
    "final_feedback": "..."
}}
"""
    response_text = call_gemini(prompt)
    evaluation = safe_parse_json(response_text, {"overall_quality": 0, "recommendation": "revise", "final_feedback": "Failed to generate evaluation."})

    return {**state, "final_evaluation": evaluation}


In [221]:

# def display_results_node(state: InterviewState) -> InterviewState:
#     """Display final report"""
#     print("\n" + "="*60)
#     print(" INTERVIEW COMPLETE - FINAL REPORT")
#     print("="*60)
#     print(f"\n Topic: {state['topic']}")
#     print("\n Questions & Feedback:")
#     for i, (q, a, f) in enumerate(zip(state['questions'], state['answers'], state['feedback']), 1):
#         print(f"\n{i}. {q}")
#         print(f"    Answer: {a}")
#         print(f"    Feedback: {f}")

#     print("\n📊 Final Evaluation:")
#     eval_data = state['final_evaluation']
#     if "error" in eval_data:
#         print(" Could not parse evaluation:", eval_data["error"])
#     else:
#         for k, v in eval_data.items():
#             print(f"   {k}: {v}")

#     with open("interview_results.json", "w") as f:
#         json.dump(state, f, indent=2)
#     print("\n Results saved to 'interview_results.json'")
#     return state


def display_results_node(state: InterviewState) -> InterviewState:
    """Display final report"""
    print("\n" + "="*60)
    print("🎯 INTERVIEW COMPLETE - FINAL REPORT")
    print("="*60)
    print(f"\n📋 Topic: {state['topic']}")
    print(f"📝 Questions Completed: {len(state['questions'])}")
    
    # Show minimal question summary first
    print(f"\n📋 QUESTIONS SUMMARY:")
    for i, q in enumerate(state['questions'], 1):
        print(f"  {i}. {q[:80]}{'...' if len(q) > 80 else ''}")
    
    # Now show all the detailed feedback
    display_all_feedback(state)
    
    # Then show the final evaluation
    print("\n" + "="*60)
    print("📈 FINAL EVALUATION")
    print("="*60)
    
    eval_data = state['final_evaluation']
    if "error" in eval_data:
        print("❌ Could not parse evaluation:", eval_data["error"])
    else:
        print(f"   Overall Quality: {eval_data.get('overall_quality', 'N/A')}/10")
        print(f"   Recommendation: {eval_data.get('recommendation', 'N/A')}")
        print(f"\n   Strengths:")
        for strength in eval_data.get('strengths', []):
            print(f"     • {strength}")
        print(f"\n   Areas for Improvement:")
        for area in eval_data.get('areas_for_improvement', []):
            print(f"     • {area}")
        print(f"\n   Final Feedback: {eval_data.get('final_feedback', 'N/A')}")

    # Save results
    with open("interview_results.json", "w") as f:
        json.dump(state, f, indent=2)
    print(f"\n💾 Results saved to 'interview_results.json'")
    
    return state

In [222]:
def display_all_feedback(state: InterviewState):
    """Display all question and answer feedback at the end in a clean format."""
    print("\n" + "="*60)
    print("📊 DETAILED FEEDBACK SUMMARY")
    print("="*60)
    
    for i, (q, a, f) in enumerate(zip(state['questions'], state['answers'], state['feedback']), 1):
        print(f"\n{'='*50}")
        print(f"QUESTION {i}: {q}")
        print(f"{'='*50}")
        print(f"💭 YOUR ANSWER: {a}")
        print(f"\n💡 QUESTION FEEDBACK (Rating: {f['question_feedback']['rating']}/10):")
        print(f"   {f['question_feedback']['feedback']}")
        print(f"\n✅ ANSWER FEEDBACK (Rating: {f['answer_feedback']['rating']}/10):")
        print(f"   {f['answer_feedback']['feedback']}")

In [223]:
def get_answer_node(state: InterviewState) -> InterviewState:
    """Get candidate answer"""
    current_q = state.get("current_question")
    if not current_q:
        raise ValueError("No current_question found in state.")

    print(f"\n{'='*50}")
    print(f"❓ QUESTION {state['step'] + 1}/{state['max_questions']}")
    print(f"{'='*50}")
    print(f"{current_q}\n")
    
    answer = input("💭 Your answer: ").strip()

    new_messages = state['messages'] + [
        {"role": "interviewer", "content": current_q},
        {"role": "candidate", "content": answer}
    ]

    return {
        **state,
        "current_answer": answer,
        "messages": new_messages,
        "questions": state['questions'] + [current_q],
        "answers": state['answers'] + [answer]
    }

In [224]:

# ---- Graph ----
def should_continue(state: InterviewState) -> str:
    return "generate_question" if state['step'] < state['max_questions'] else "final_evaluation"


In [225]:

builder = StateGraph(InterviewState)
builder.add_node("setup", setup_node)
builder.add_node("get_answer", get_answer_node)
builder.add_node("evaluate_question", evaluate_question_node)
builder.add_node("generate_question", generate_question_node)
builder.add_node("final_evaluation", final_evaluation_node)
builder.add_node("display_results", display_results_node)

builder.set_entry_point("setup")
builder.add_edge("setup", "get_answer")
builder.add_edge("get_answer", "evaluate_question")
builder.add_conditional_edges(
    "evaluate_question",
    should_continue,
    {"generate_question": "generate_question", "final_evaluation": "final_evaluation"}
)
builder.add_edge("generate_question", "get_answer")
builder.add_edge("final_evaluation", "display_results")
builder.add_edge("display_results", END)

interview_graph = builder.compile()


In [226]:
# Run this cell first to upload your CV
cv_data = upload_cv_interactively()

 Welcome to the AI Interviewer (Question Evaluation Mode)!
Please upload your CV/Resume


In [ ]:

# ---- Run ----
if __name__ == "__main__":

# Run this cell after uploading CV to start the interview
    if cv_data and cv_data.get("cv_content"):
        initial_state = {
            "topic": cv_data["topic"],
            "content": [],
            "questions": [],
            "answers": [],
            "feedback": [],
            "current_question": None,
            "current_answer": None,
            "step": 0,
            "max_questions": 3,
            "final_evaluation": None,
            "messages": [],
            "question_type": "broad_followup",
            "cv_content": cv_data["cv_content"],
            "cv_filename": cv_data["cv_filename"]
        }
        
        final_state = interview_graph.invoke(initial_state)
    else:
        print("No CV data found. Please upload a CV first.")

 Welcome to the AI Interviewer (Question Evaluation Mode)!
🎯 Interview focus: Mobile application development (React Native & Jetpack Compose)
Choose question style:
1. Broad, follow-up questions (general, builds on previous answers)
2. Narrow, follow-up questions (specific, probes details from previous answers)
3. Broad, non-follow-up questions (general, new topic aspects)
4. Narrow, non-follow-up questions (specific, new topic aspects)


Enter choice (1-4):  2



❓ QUESTION 1/3
In your experience developing cross-platform mobile applications with React Native, and native Android applications with Jetpack Compose (such as the 'Smart Beauty' app), you've focused on UI/UX implementation, responsiveness, and performance optimization. Can you describe a specific instance where you had to significantly optimize the performance or smooth UI interactions of an application using either React Native or Jetpack Compose, detailing the problem you faced, the specific tools and techniques you employed, and the measurable impact of your solution?



💭 Your answer:  i cant say



❓ QUESTION 2/3
Based on your understanding, what's the fundamental difference in the type of application development React Native enables compared to Jetpack Compose?



💭 Your answer:  React native uses js which then interacts with java or kotlin but jc directly deALS WITH JAVA AND KOTLINA
